In [8]:
#########################################################################
# busca todos os arquivos período explícito sem especificar a data
#########################################################################

import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time, getpass
from bs4 import  BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd

def build_path(subfolder):
    current_folder = os.getcwd()
    folderpath = os.path.join(current_folder, subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): os.makedirs(folderpath)
    return folderpath

build_path('arquivos')

# login page

def DigitaNaCaixaId(idCaixa, Texto):
  global driver
  Caixa = driver.find_element_by_id(idCaixa)
  Caixa.clear()
  Caixa.send_keys(Texto)

def UsuarioDigitaLoginEsenha():
  global LoginUsuario, Senha
  print('Login do SEI:')
  LoginUsuario = input()
  Senha = getpass.getpass("Digite sua senha (não vai aparecer na tela): ")

def FazLogin():
  global driver, WebEspera
  global LoginUsuario, Senha
  url = 'https://sip.sgb.gov.br/sip/login.php?sigla_orgao_sistema=CPRM&sigla_sistema=SEI&infra_url=L3NlaS8='
  driver = webdriver.Edge('C:\Program Files\Edge\msedgedriver.exe')
  driver.implicitly_wait(10)
  driver.get(url)
  driver.maximize_window()
  WebEspera = WebDriverWait(driver, 20)
  
  DigitaNaCaixaId('txtUsuario', LoginUsuario)
  time.sleep(1)
  DigitaNaCaixaId('pwdSenha', Senha)
  time.sleep(1)

  # ContagemRegressiva(TempoEsperaLogin)

  LoginEnter = driver.find_element("xpath", '//*[@id="sbmLogin"]')
  LoginEnter.click()
  time.sleep(2)
# Fim FazLogin()

UsuarioDigitaLoginEsenha()
FazLogin()
print('Seu acesso foi realizado com sucesso!')


# home page

searching = driver.find_element("xpath", '//*[@id="main-menu"]/li[5]/a')
searching.click()

# search page

doc_type = driver.find_element("xpath", '//*[@id="selSeriePesquisa"]')
doc_date_explicit = driver.find_element("xpath", '//*[@id="optPeriodoExplicito"]')
sub_button = driver.find_element("xpath", '//*[@id="sbmPesquisar"]')

doc_type.send_keys('REMA - Empréstimo de Materiais ou Ex. Geológicos')
doc_date_explicit.click()
sub_button.click()


#################################################
def get_files():

    original_window = driver.current_window_handle 
    page_docs_search = driver.find_element_by_xpath('//*[@id="conteudo"]')  # Corrigido de "driver.find_element" para "driver.find_element_by_xpath"
    page_docs = page_docs_search.text.split('Atividade Fim: ')[1:]

    file_name = []
    for i in range(len(page_docs)):
        a = page_docs[i].split('(REMA - Empréstimo de Materiais ou Ex. Geológicos) ')[1].split('\n1')[0]
        file_name.append(a)

    links = []
    for i in range(1, len(page_docs)+1):
        a = driver.find_element_by_xpath(f'//*[@id="conteudo"]/table[{i}]/tbody/tr[1]/td[1]/a[2]')
        link = a.get_attribute('href')
        links.append(link)
    
    for i in links:
        name = file_name[links.index(i)]  # Corrigido para encontrar o índice correto
        driver.execute_script("window.open('');")  # Alterado para a abertura de uma nova aba
        driver.switch_to.window(driver.window_handles[1])  # Mudado para o índice 1
        driver.get(i)
        with open(f'./arquivos/processo{name}.html', 'w', encoding='iso-8859-1') as file:  # Corrigido a string de formatação e a sintaxe de abertura de arquivo
            file.write(driver.page_source)        
        driver.close()
        driver.switch_to.window(original_window) 
  

#################################################

# pagination 

pages = driver.find_element(by=By.CLASS_NAME, value="paginas")
list_pages = pages.text.split(' ')[:-1]

get_files()

for i in range(1, len(list_pages)):
    
    try:
        next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/a[%s]' %i)
        next_page.click()
        time.sleep(6)
        get_files()
    except (NameError, TypeError):
        pass
    time.sleep(3)  
    

driver.close()
driver.quit()

### Rrtirar dados do html para a planilha

pasta = f'arquivos'
arquivo = os.listdir(pasta)
fname = [pasta +'/'+ arq for arq in arquivo if arq.endswith(".html")]


lista_df=[]

for i in fname:

    with open(i, "r", encoding="iso-8859-1") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    tags = [tag for tag in soup.find("div", id="conteudo").children if len(tag.text.strip()) > 0 and not re.match(r"^\d+\.", tag.text.strip())]
    # Cada HTML, um dicionário ordenado
    dict_series = OrderedDict()

    for index in range(len(tags)):
        tag = tags[index]
        
        if tag.name == "b":
            key = tag.text.strip().rstrip(":")
            value = tags[index + 1].text.strip()
            
            dict_series[key] = value


    # Empilhar todos os dicionários para criar o df e interpretar os dtypes
    df = (
        pd.DataFrame([dict_series])
            .apply(lambda x: pd.to_numeric(x.str.replace(",", "."), errors="ignore"))
            .apply(lambda x: x.replace("Sim", True).replace("Não", False))        
    )
    lista_df.append(df)

DataFrame=pd.concat(lista_df,ignore_index=index)
DataFrame

DataFrame.to_excel('C:/Users/Rosana/Desktop/SEI/REMA.xlsx', index=False)

print("Concluído")

Login do SEI:


 daiana.sales
Digite sua senha (não vai aparecer na tela):  ········


Seu acesso foi realizado com sucesso!
Concluído
